In [1]:
from polygon import RESTClient
from typing import cast
from urllib3 import HTTPResponse
from config import POLYGON_API_KEY
import json
from datetime import datetime
import time

In [2]:
# client = RESTClient(POLYGON_API_KEY)

# aggs = cast(
#     HTTPResponse,
#     client.get_aggs(
#         "JPM",
#         1,
#         "day",
#         "2022-09-14",
#         "2022-09-15",
#         raw=True,
#     ),
# )

In [6]:
time.mktime(datetime(2022, 9, 15, 11, 0).timetuple())*1e9

1.663254e+18

In [40]:
client = RESTClient(POLYGON_API_KEY)

quotes_aggs = cast(
    HTTPResponse,
    client.list_quotes(
        ticker="JPM",
        timestamp_gte=datetime(2022, 9, 15, 11, 0),
        timestamp_lte=datetime(2022, 9, 15, 11, 1),
        limit=50000,
        raw=True,
    ),
)

trades_aggs = cast(
    HTTPResponse,
    client.list_trades(
        ticker="JPM",
        timestamp_gte=datetime(2022, 9, 15, 11, 0),
        timestamp_lte=datetime(2022, 9, 15, 11, 1),
        limit=50000,
        raw=True,
    ),
)

In [48]:
print(aggs.geturl())
print(aggs.status)
quotes_aggs_res = json.loads(quotes_aggs.data.decode("utf-8"))['results']
trades_aggs_res = json.loads(trades_aggs.data.decode("utf-8"))['results']
print(f"{len(quotes_aggs_res)} quotes")
print(f"{len(trades_aggs_res)} trades")

https://api.polygon.io/v3/quotes/JPM
200
3380 quotes
262 trades


In [49]:
print(json.dumps(quotes_aggs_res[:5], indent=4))

[
    {
        "ask_exchange": 12,
        "ask_price": 117.48,
        "ask_size": 1,
        "bid_exchange": 11,
        "bid_price": 117.46,
        "bid_size": 1,
        "conditions": [
            1,
            81
        ],
        "indicators": [
            1
        ],
        "participant_timestamp": 1663254059976813288,
        "sequence_number": 16646889,
        "sip_timestamp": 1663254059977052928,
        "tape": 1
    },
    {
        "ask_exchange": 12,
        "ask_price": 117.48,
        "ask_size": 1,
        "bid_exchange": 21,
        "bid_price": 117.46,
        "bid_size": 2,
        "conditions": [
            1,
            81
        ],
        "indicators": [
            1
        ],
        "participant_timestamp": 1663254059976557268,
        "sequence_number": 16646885,
        "sip_timestamp": 1663254059976802304,
        "tape": 1
    },
    {
        "ask_exchange": 12,
        "ask_price": 117.48,
        "ask_size": 1,
        "bid_exchange": 11,


In [50]:
print(json.dumps(trades_aggs_res[:5], indent=4))

[
    {
        "conditions": [
            32,
            37
        ],
        "exchange": 4,
        "id": "79371907451982",
        "participant_timestamp": 1663254057807000000,
        "price": 117.45,
        "sequence_number": 738010,
        "sip_timestamp": 1663254057801309440,
        "size": 10,
        "tape": 1,
        "trf_id": 10,
        "trf_timestamp": 1663254057801290413
    },
    {
        "conditions": [
            37
        ],
        "exchange": 4,
        "id": "71679970724676",
        "participant_timestamp": 1663254057438000000,
        "price": 117.4549,
        "sequence_number": 737977,
        "sip_timestamp": 1663254057440791040,
        "size": 8,
        "tape": 1,
        "trf_id": 12,
        "trf_timestamp": 1663254057440438318
    },
    {
        "conditions": [
            37
        ],
        "exchange": 4,
        "id": "71679970722872",
        "participant_timestamp": 1663254056955561299,
        "price": 117.46,
        "sequence_numbe